# Sentiment Analysis with BERT

> TL;DR In this notebook, we fine-tune BERT for sentiment analysis. We perform the required dataset preparation using Pandas, and then we do the text preprocessing and build a Sentiment Classifier using the Transformers library by Hugging Face as well as Pytorch.

Goal (official): To get a model that can classify text based on seniment, and - hopefully - performs well on neutral text, which can be problematic for some models.

Goal (actual): To understand the entire pipeline better, become more skilled, and perhaps get a few ideas on how to perform some personal project ideas that I have. Also: For GPUs to go "Brrrrrrrrrrrrrr". (duh)

**Multiple resources were used in creating this notebook, but mostly, it's based on [this notebook](https://github.com/curiousily/Getting-Things-Done-with-Pytorch/blob/master/08.sentiment-analysis-with-bert.ipynb), made by [Venelin Valkov](https://www.youtube.com/@venelin_valkov). I don't like copy/pasting code that I don't understand, so I did not include some of the fancier sections from that notebook here. Other sources were used as well, including ChatGPT and other online sources.**

In [1]:
!nvidia-smi

Tue Apr 11 13:16:45 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 531.41                 Driver Version: 531.41       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                      TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 960M       WDDM | 00000000:02:00.0 Off |                  N/A |
| N/A    0C    P0               N/A /  N/A|      0MiB /  4096MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Mon_Oct_24_19:40:05_Pacific_Daylight_Time_2022
Cuda compilation tools, release 12.0, V12.0.76
Build cuda_12.0.r12.0/compiler.31968024_0


In [3]:
import torch
torch.cuda.is_available()

True

## Setup

We'll need to make sure Transformers is installed before we start:

In [4]:
!pip install -q transformers


[notice] A new release of pip available: 22.3 -> 23.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


And we'll also need to import the following:

In [5]:
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup

import torch
from torch import nn, optim
from torch.nn.utils import clip_grad_norm_
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.utils import shuffle

import numpy as np
import pandas as pd

from collections import defaultdict  
# a dictionary-like object that initializes nonexistent keys, if called, to a default pre-chosen value

from tqdm import tqdm
# isn't it nice when you can personally sees the GPUs gp "Brrrrr"?

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

/home/ubuntu/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


device(type='cuda', index=0)

We can use the watermark extension for Jupyter notebooks here to view the versions of th most important libraries we'll be using. In case of a code error, version incompatibility is one of the usual suspects.

In [6]:
!pip install -q -U watermark 
# -q is for "quiet". This means it won't output anything 

In [5]:
# reload the watermark extension for Jupyter
%reload_ext watermark

# use watermark to display the version info for the specified packages
%watermark -v -p numpy,pandas,torch,transformers

Python implementation: CPython
Python version       : 3.8.5
IPython version      : 8.11.0

numpy       : 1.23.5
pandas      : 1.1.5
torch       : 1.7.1
transformers: 4.1.1



## Data Exploration

In this section, we will:

1. Download the [selected dataset](https://www.kaggle.com/datasets/jillanisofttech/amazon-product-reviews) from Kaggle using the kaggle API.
2. Load our dataset using Pandas.
3. View the basic structure of the dataset: `head()`\\`sample()`, `info()`, etc.
4. Check for missing data, and for any needed changes. (here: heavy imbalance between sentiments was found).

Let's begin by downloading the [selected dataset](https://www.kaggle.com/datasets/jillanisofttech/amazon-product-reviews) from Kaggle using the kaggle API.

In [8]:
!pip install -q -U kaggle


[notice] A new release of pip available: 22.3 -> 23.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


We follow the instructions found on [Kaggle's website](https://www.kaggle.com/docs/api) to get our API token and authenticate it. We should know there was no issue with the authentication if we can run `import kaggle` successfully without errors.

In [9]:
import kaggle

Huh, seems like it works. This coding stuff must be real.

Now, let's download our dataset. Using the convenient **"Copy API command"** option available on every Kaggle dataset page, we apply:

In [7]:
!kaggle datasets download -d jillanisofttech/amazon-product-reviews


  0%|          | 0.00/115M [00:00<?, ?B/s]
  1%|          | 1.00M/115M [00:00<00:26, 4.46MB/s]
  2%|1         | 2.00M/115M [00:00<00:19, 5.93MB/s]
  3%|2         | 3.00M/115M [00:00<00:17, 6.56MB/s]
  3%|3         | 4.00M/115M [00:00<00:15, 7.35MB/s]
  4%|4         | 5.00M/115M [00:00<00:14, 7.87MB/s]
  6%|6         | 7.00M/115M [00:00<00:11, 9.68MB/s]
  8%|7         | 9.00M/115M [00:01<00:10, 10.9MB/s]
 10%|9         | 11.0M/115M [00:01<00:09, 11.5MB/s]
 11%|#1        | 13.0M/115M [00:01<00:09, 11.6MB/s]
 13%|#3        | 15.0M/115M [00:01<00:08, 13.0MB/s]
 15%|#4        | 17.0M/115M [00:01<00:07, 13.0MB/s]
 17%|#6        | 19.0M/115M [00:02<00:11, 8.67MB/s]
 18%|#8        | 21.0M/115M [00:02<00:12, 7.94MB/s]
 20%|##        | 23.0M/115M [00:02<00:09, 9.71MB/s]
 23%|##2       | 26.0M/115M [00:02<00:07, 13.0MB/s]
 24%|##4       | 28.0M/115M [00:03<00:09, 9.72MB/s]
 28%|##7       | 32.0M/115M [00:03<00:06, 13.1MB/s]
 31%|###       | 35.0M/115M [00:03<00:06, 12.3MB/s]
 33%|###3      | 38.

The above code needs a `!` at the beginning, because what we installed above and are using to interact with the Kaggle API is actually a [CLI](https://en.wikipedia.org/wiki/Command-line_interface) tool.

Running the above cell tells us that our zip file `amazon-product-reviews.zip` has been downloaded to the current directory.

Now, let's unzip this zip file.

I don't know/remember how to unzip using Python. Fortunately, Stack Overflow is a thing.

In [9]:
import zipfile
with zipfile.ZipFile('amazon-product-reviews.zip', 'r') as zip_file:
    zip_file.extractall('')

In [10]:
import os
os.remove('amazon-product-reviews.zip')

We can see that we now have a `Reviews.csv` file in our directory. Let's use Pandas to load it and take a look at it:

In [13]:
import pandas as pd
df = pd.read_csv('Reviews.csv')
print(df.shape)
df.sample(5)

(568454, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
119015,119016,B003YF1188,A3MB83ALNB3O4Z,Ann,0,0,5,1350432000,I am a fan of Stonewall Kitchen!,I was so happy that Amazon carries this produc...
120975,120976,B001EQ57KW,A2MM5OQCXV4BQ1,GAD,0,0,5,1338422400,"Good for snacking, great for baking!","First, I love the fact that Go Raw processes t..."
306052,306053,B002R89LOE,AY1EF0GOH80EK,Natasha Stryker,7,7,2,1276819200,Am I just crazy?,I feel like I am taking crazy pills when I rea...
485495,485496,B001RVFERK,AJFXMVJTGGHTY,"Wade Osborne ""Wade Osborne""",0,0,4,1312761600,great chips,Pop Chips are the best chips I've had that are...
490320,490321,B001E5DZJ8,A334K3EPD2H467,Sarah Norman,0,0,5,1314662400,Tastes great,These taste great. I add 1 or 2 cubes to the ...


All we really care about here are two columns: "Text" and "Score", as those are the ins and outs of the model.

"Summary" and "HelpfulnessDenominator" can be useful to take a look at too, for a future project perhaps.

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568454 entries, 0 to 568453
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   Id                      568454 non-null  int64 
 1   ProductId               568454 non-null  object
 2   UserId                  568454 non-null  object
 3   ProfileName             568438 non-null  object
 4   HelpfulnessNumerator    568454 non-null  int64 
 5   HelpfulnessDenominator  568454 non-null  int64 
 6   Score                   568454 non-null  int64 
 7   Time                    568454 non-null  int64 
 8   Summary                 568427 non-null  object
 9   Text                    568454 non-null  object
dtypes: int64(5), object(5)
memory usage: 43.4+ MB


It looks like we have a fantastically complete dataset here! ~570K rows that are almost all complete, with a handful of missing values in the entire thing. Cool.

The two columns we care about the most, `Text` and `Score` are complete as well. We get to live to fight (or clean data) another day.

I'm still interested in filtering out those rows with null values in them in the "Summary" column, however. Maybe we could use that column later, so I'd like to drop these rows now:

In [15]:
df = df.dropna(axis='index', how='all', subset=['Summary'])

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 568427 entries, 0 to 568453
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   Id                      568427 non-null  int64 
 1   ProductId               568427 non-null  object
 2   UserId                  568427 non-null  object
 3   ProfileName             568411 non-null  object
 4   HelpfulnessNumerator    568427 non-null  int64 
 5   HelpfulnessDenominator  568427 non-null  int64 
 6   Score                   568427 non-null  int64 
 7   Time                    568427 non-null  int64 
 8   Summary                 568427 non-null  object
 9   Text                    568427 non-null  object
dtypes: int64(5), object(5)
memory usage: 47.7+ MB


With a large datset such as this one, it's hard to avoid redundancy. People will repeat themselves, especially with shorter reviews. Let's see how many unique reviews we've got:

In [17]:
len(df.Text.unique())

393576

Some 175K reviews are duplicates here, which is roughly the third of the entire dataset. Let's drop these now and save ourselves some precious GPU training hours:

In [18]:
df.drop_duplicates(subset='Text', keep='first', inplace=True)

In [19]:
len(df)

393576

And now, let's count the remaining reviews by score:

In [20]:
df.Score.value_counts()

5    250716
4     56042
1     36275
3     29752
2     20791
Name: Score, dtype: int64

It seems like most of the reviews are positive. Will this is certainly good for merchants, it's bad news for models that want to train hard to learn those better weights and biases. We'll deal with this imbalance in the next section.

Let's now move to the next section, where we will get our data ready to be fed into Bert.

## Data Preprocessing

In this section, we will:

1. Create a function to turn the ratings which are currently numbers into 3 sentiment classes. 
2. Create a balanced dataset by making sure we have roughly equal neutral scores to +ve and -ve ones.
3. Decide on a max. sequence length (in tokens) and trim longer texts to balance training cost vs. accuracy.
4. Create our Pytorch Dataset class.
5. Decide what batch size to use and create our Pytorch Dataloader class/function.
6. Split the dataset into training, validation and test datasets, and create dataloaders for them.
7. Examine one of our batches before moving on to the training section.

Let's begin by defining a `score2sentiment` function that will do as its name suggests. We'll define the scores 5 and 4 as being positive, 3 as being neutral, and 2 and 1 as being negative. You may want to stick with these definitions, or you might want to change them or add more classes. You would only need to keep in mind that any changes to this decision will carry out and require subsequent changes down the line. So we apply here:

In [21]:
def score2sentiment(score):
    if score < 3 :
        return 'negative'
    elif score > 3 :
        return 'positive'
    else:
        return 'neutral'

Let's apply this transformation to the entire dataset:

In [22]:
df['Sentiment'] = df.Score.apply(score2sentiment)

Now would be a good time to look at how well the data is distributed across sentiments:

In [23]:
df['Sentiment'].value_counts()

positive    306758
negative     57066
neutral      29752
Name: Sentiment, dtype: int64

I don't need all of this data for my current use case, so balancing the dataset by removing most of it is a luxury that I can afford:

In [24]:
positive_df = df[(df.Sentiment == 'positive')]
negative_df = df[(df.Sentiment == 'negative')]
neutral_df = df[(df.Sentiment == 'neutral')]

balanced_df = pd.concat([positive_df.head(29_750), negative_df.head(29_750), neutral_df.head(29_750)])

In [25]:
balanced_df.Sentiment.value_counts()

positive    29750
negative    29750
neutral     29750
Name: Sentiment, dtype: int64

Now, going back to our to do list, we see that we will now be deciding on a maximum sequence length (in tokens), and that we need to trim longer texts to balance training cost vs. accuracy.

Let's start by loading our tokenizer:

In [26]:
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

We will now tokenize our dataset, but without setting any maximum length, in order to see how much of the dataset is longer than the maximum allowed sequence length we have in our model here:

In [56]:
tokens_lengths = []

for review in tqdm(balanced_df.Text):
    tokens = tokenizer.encode_plus(review,
                                   add_special_tokens=True,
                                   return_length=True)

    tokens_lengths.append(tokens.length)

100%|██████████| 89250/89250 [01:23<00:00, 1064.05it/s]


In [28]:
great_tokens_lengths = [x for x in tokens_lengths if x > 512]

In [29]:
(len(great_tokens_lengths) / len(tokens_lengths)) * 100

1.4980392156862745

As we can see, only 1.5% of the reviews are longer than 512 tokens, which is the limit for Bert. That means that only 1.5% of the dataset would be affected if we trim longer inputs. So we can safely decide on a max. len. of 512.

In [30]:
MAX_LEN = 512

Next, we will create our Pytorch Dataset class, followed by a small function that turns any dataset we pass to it into a DataLoader object.

In [32]:
from torch.utils.data import Dataset

sentiment2number = {"negative" : 0, "neutral" : 1, "positive" : 2}

class AmazonReviewDataset(Dataset):
    
    def __init__(self, df, tokenizer,):
        self.df = df
        self.reviews = df.Text.to_numpy()
        self.sentiments = df.Sentiment.to_numpy()
        self.tokenizer = tokenizer
    
    # We also need to define __len__() and __getitem__():
    
    def __len__(self):
        return len(self.df['Id'])
    
    def __getitem__(self, idx):
        
        review = self.reviews[idx]
        sentiment = self.sentiments[idx]
        
        sentiment = sentiment2number[sentiment]
        sentiment = torch.tensor(sentiment, dtype=torch.long)
        
        tokens = self.tokenizer.encode_plus(review,
                                             add_special_tokens=True,
                                             max_length=512,
                                             padding='max_length',
                                             truncation=True,
                                             return_token_type_ids=False,
                                             return_tensors='pt')

        return {"review": review, "input_ids": tokens['input_ids'].flatten(), 
                "attention_mask": tokens['attention_mask'].flatten(), "sentiment": sentiment}
    
# we must first transform sentiment to a number, then to type torch.long as that is required for classification
# .flatten() turns our ...

In [33]:
from torch.utils.data import DataLoader

def create_dataloader(dataset, tokenizer):
    processed_ds = AmazonReviewDataset(dataset, tokenizer)
    
    return DataLoader(processed_ds, batch_size=16, shuffle=True)

I tried a few numbers and settled with 16 as my batch size, but this will be experimental, and sometimes restricted based on the available hardware for training.

Let's now shuffle the dataset, and then split it into training, validation and test datasets, and create dataloaders for these.

In [34]:
shuffled_df = shuffle(balanced_df)

train_df, val_df = train_test_split(shuffled_df, train_size=0.75)
val_df, test_df = train_test_split(val_df, test_size=0.5)

In [35]:
train_dl = create_dataloader(train_df, tokenizer)
val_dl = create_dataloader(val_df, tokenizer)
test_dl = create_dataloader(test_df, tokenizer)

Finally, let's have a look at an example batch from our training data loader:

In [36]:
batch = next(iter(train_dl))

print(batch.keys())
print(batch['input_ids'].shape)
print(batch['attention_mask'].shape)
print(batch['sentiment'].shape)

dict_keys(['review', 'input_ids', 'attention_mask', 'sentiment'])
torch.Size([16, 512])
torch.Size([16, 512])
torch.Size([16])


## Defining Training Process

Instead of using the Sentiment Analysis helper built for BERT which comes with the Transformers library, we'll use the basic `BertModel` and build our sentiment classifier on top of it.

So in this section, we will:

1. Load the base model (cased).
2. run it on the sample text from previous section.
3. Build a Sentiment Analysis wrapper around BERT using Pytorch.
4. Show pre-trained BERT's ability to classify our sample text.
5. Decide what loss function, optimizer, scheduler to use as well as the rest of the hyper-parameters.
6. Create our training epoch function.
7. Create our inference function.
8. Finish with our training loop.
9. View progress during fine-tuning.

In [37]:
bert_model = BertModel.from_pretrained('bert-base-cased', return_dict=False)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


And try to use it on the encoding of our sample text:

In [38]:
sample_input_ids = torch.reshape(batch["input_ids"][0], (1, 512))
sample_attention_mask = torch.reshape(batch["attention_mask"][0], (1, 512))

sample = {'input_ids':sample_input_ids, 'attention_mask': sample_attention_mask}
raw, pooled = bert_model(**sample)

We can see that we had to reshape the tensors before feeding them into our model. To understand why, we need to consider the following:

Shape of the input tensors **without** using `.flatten()`: `(batch_size, 1, sequence_length)` = `(16, 1, 512)`.

Shape **with** using `.flatten()`: `(batch_size, sequence_length)` = `(16, 512)`.

We can use `.flatten()` within `DataLoader`, presumably as it already has information about the batch size and preprocesses accordingly.

We shouldn't use `.flatten()` for casual inference like we're doing now, as the model expects an input of a fixed length and shape (?) (Here: 512, since this is a BERT-model). 

In [39]:
raw

tensor([[[ 0.4075,  0.2071,  0.1190,  ..., -0.4108,  0.0408, -0.0849],
         [ 0.2758, -0.6349,  0.2261,  ..., -0.2547,  0.1820,  0.0482],
         [ 0.5432, -0.6583,  0.1456,  ...,  0.6100,  0.6814, -0.2100],
         ...,
         [ 0.1340,  0.2772, -0.2694,  ...,  0.1435,  0.1378,  0.2156],
         [ 0.1539,  0.2833, -0.3036,  ...,  0.1528,  0.1631,  0.2584],
         [ 0.0829,  0.2475, -0.2725,  ...,  0.1568,  0.2055,  0.2240]]],
       grad_fn=<NativeLayerNormBackward0>)

The raw output of the model, `raw`, is a sequence of hidden states of the last layer of the model. Obtaining the pooled output `pooled` is done by applying pooling on the last hidden state `raw`.

You can think of `pooled` as a summary of the content, according to Bert. Let's look at the shape of the output:

In [40]:
raw.shape, pooled.shape

(torch.Size([1, 512, 768]), torch.Size([1, 768]))

We can now move on to building our Sentiment Analysis wrapper around BERT using Pytorch:

In [41]:
class SentimentClassifier(nn.Module):
    
    def __init__(self, num_classes):
        super(SentimentClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-cased', return_dict=False)
        self.drop = nn.Dropout(p=0.25)
        self.output_layer = nn.Linear(768, num_classes)
        # self.softmax = nn.Softmax(dim=1) # Why 1? to apply it among classes and not batches. Tell me more.
        
    def forward(self, input_ids, attention_mask):
        raw, pooled = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled = self.drop(pooled)
        fine_tuned = self.output_layer(pooled)
        # classified = self.softmax(fine_tuned)
        
        return fine_tuned

We use a dropout layer for some regularization and a fully-connected layer for our output. **Note that we're returning the raw output of the last layer since that is required for the cross-entropy loss function in PyTorch to work.**

Let's now initialize the model, move it to the GPU and show pre-trained Bert's ability to classify our sample text:

In [42]:
num_classes = len(df.Sentiment.unique())

classifier = SentimentClassifier(num_classes)
classifier = classifier.to(device)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [43]:
number2sentiment = {0: "negative", 1: "neutral", 2: "positive"}

sample = {'input_ids':sample_input_ids.to(device), 'attention_mask': sample_attention_mask.to(device)}

pooled = classifier(**sample)
probs = F.softmax(pooled, dim=1)

print(batch['review'][0])
print(batch['sentiment'][0])

result = number2sentiment[torch.argmax(probs).item()]

result

BEWARE  I recieved my box of 25 assorted Nonnie's Biscotti from Amazon . Every single biscotti was broken in 1-3 pieces. I am disgusted completely as these were to be favors of a wedding . I will never buy from this company again.  BEWARE
tensor(0)


'neutral'

In [44]:
# output.detach_()
# del output
# torch.cuda.empty_cache()

In [45]:
# torch.cuda.memory_allocated()

Finally, we will now decide what loss function, optimizer and scheduler to use, as well as the rest of our hyper-parameters.

We need to define:

1. Number of epochs.
2. Learning rate.
3. Our optimizer.
4. Our scheduler.
5. Our loss function.

In [46]:
EPOCHS = 3
lr = 5e-5

num_steps = EPOCHS * len(train_dl)

optimizer = optim.AdamW(classifier.parameters(), lr=lr)

scheduler = get_linear_schedule_with_warmup(optimizer=optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=num_steps)

loss_fn = nn.CrossEntropyLoss().to(device)

How come those values?

Well, the number of epochs and the learning rate are among the recommended values in the original [Bert paper](https://arxiv.org/abs/1810.04805).

The loss function is Cross Entropy, which is commonly used for Sentiment Analysis and is suitable because it always outputs values that add up to 1, which is exactly what all probabilities add up to in any scenario. That means we can always use it for problems with a defined number of classes.

The optimizer and scheduler are the ones used in the [original version](https://github.com/curiousily/Getting-Things-Done-with-Pytorch/blob/master/08.sentiment-analysis-with-bert.ipynb) of this notebook, by [Venelin Valkov](https://www.youtube.com/@venelin_valkov).

### Training

We are finally ready to create our training epoch function, followed by our inference function, and our training loop:

In [47]:
def train_one_epoch(model, dataloader, loss_fn, optimizer, scheduler, device):
    # 1. Set the model in training mode
    model = model.train()
    
    # 2. Initialize the variables for tracking loss and correct predictions
    loss_history = []
    correct_predictions = 0
    
    # 3. Iterate over the batches in the data loader
    for batch in tqdm(dataloader):
        
        # a. Move the input and target tensors to the specified device
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        targets = batch['sentiment'].to(device)
        
        # b. Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        # only returns the processed 'pooled', as we defined in our S.C. class
        
        values, predictions = torch.max(outputs, dim=1) 
        # specifying the dim. changes the returns of torch.max()
        
        loss = loss_fn(outputs, targets)
        
        # c. Update the loss and correct predictions variables
        loss_history.append(loss.item())
        correct_predictions += torch.sum(predictions == targets)
        # torch compares values for numbers, regardless of dtype
        
        # d. Backward pass
        loss.backward()
        clip_grad_norm_(parameters=model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        
    # 4. Compute the accuracy and average loss
    accuracy = correct_predictions.double()/(len(dataloader) * 16)
    average_loss = np.mean(loss_history)
        
    return accuracy, average_loss

The training loop should look familiar, except for two things: The scheduler gets called every time a batch is fed to the model, an we're avoiding exploding gradients by clipping the gradients of the model using [clip_grad_norm_](https://pytorch.org/docs/stable/nn.html#clip-grad-norm).

Let's now write the evaluation loop:

In [48]:
def evaluate_model(model, dataloader, loss_fn, device):
    # 1. Set the model in evaluation mode
    model = model.eval()
    
    # 2. Initialize the variables for tracking loss and correct predictions
    loss_history = []
    correct_predictions = 0
    
    with torch.no_grad():
        # 3. Iterate over the batches in the data loader
        for batch in tqdm(dataloader):

            # a. Move the input and target tensors to the specified device
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            targets = batch['sentiment'].to(device)

            # b. Forward pass
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)        
            values, predictions = torch.max(outputs, dim=1)        
            loss = loss_fn(outputs, targets)

            # c. Update the loss and correct predictions variables
            loss_history.append(loss.item())
            correct_predictions += torch.sum(predictions == targets)

        # 4. Compute the accuracy and average loss
        accuracy = correct_predictions.double()/(len(dataloader) * 16)
        average_loss = np.mean(loss_history)

    return accuracy, average_loss

Using those two, we can write our training loop. We'll also store the training history:

In [49]:
# 1.a) initialize a dict. with default values that are lists
history = defaultdict(list)

# 1.b) store the highest val. acc. seen so far
best_accuracy = 0

# 2. loop through epochs
for epoch in range(EPOCHS):

    # a. Print the current epoch number
    print(f"Epoch {epoch + 1} out of {EPOCHS}.")
    print('-' * 10)

    # b. Train the model and print output
    train_acc, train_loss = train_one_epoch(classifier, train_dl, loss_fn, optimizer, scheduler, device)
    print(f"Accuracy on training dataset: {train_acc}")
    print(f"Loss on training dataset: {train_loss}")
    print('-' * 5)

    # c. Evaluate on the validation dataset and print output
    val_acc, val_loss = evaluate_model(classifier, val_dl, loss_fn, device)
    print(f"Accuracy on validation dataset: {val_acc}")
    print(f"Loss on validation dataset: {val_loss}")
    print('-' * 5)

    # d. Document the training and validation accuracy and loss in history
    history['train_acc'].append(train_acc)
    history['train_loss'].append(train_loss)
    history['val_acc'].append(val_acc)
    history['val_loss'].append(val_loss)

    # If current val_acc > best_accuracy, model state is saved to file
    if val_acc > best_accuracy:
        torch.save(classifier.state_dict(), 'best_model_state.bin')
        best_accuracy = val_acc

Epoch 1 out of 3.
----------


100%|██████████| 4184/4184 [39:46<00:00,  1.75it/s]


Accuracy on training dataset: 0.7467286089866156
Loss on training dataset: 0.5931135959189606
-----


100%|██████████| 698/698 [02:30<00:00,  4.65it/s]


Accuracy on validation dataset: 0.7821454154727794
Loss on validation dataset: 0.53433882863525
-----
Epoch 2 out of 3.
----------


...

Accuracy on validation dataset: 0.7933381088825214
Loss on validation dataset: 0.4978796033653028
-----
Epoch 3 out of 3.
----------


100%|██████████| 4184/4184 [39:53<00:00,  1.75it/s]


Accuracy on training dataset: 0.9087595602294455
Loss on training dataset: 0.25616456334308646
-----


100%|██████████| 698/698 [02:30<00:00,  4.64it/s]

Accuracy on validation dataset: 0.7905623209169054
Loss on validation dataset: 0.6446215840600922
-----


## Evaluation

Here, we will:
1. Write a function to test how well the model generalizes, by using the test dataset, which is data it didn't train on.
2. Evaluate the model's performance using classification report.
3. Try it on raw input.

In [54]:
def get_predictions_from_dl(model, dataloader):
    
    # 1. Set the model to evaluation mode
    model = model.eval()
    
    # 2. Initialize the lists for ins, true_outs, preds and confidence 
    reviews = []
    predictions = []
    confidence_levels = []
    true_sentiments = []
    
    with torch.no_grad():
        # 3. Iterate over the batches in the data loader
        for batch in tqdm(dataloader):
            # a. Move the model input tensors to the specified device,
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            # keeping the ins & outs of the dataset unmoved
            texts = batch['review']
            # using a different variable name to not overwrite the list
            sentiments = batch['sentiment']

            # b. Forward pass
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            vals, preds = torch.max(outputs, dim=1)

            # c. Get confidence levels with softmax 
            confidence_lvls = F.softmax(outputs, dim=1)

            # d. Update the lists
            reviews.extend(texts)
            predictions.extend(preds)
            confidence_levels.extend(confidence_lvls)
            true_sentiments.extend(sentiments)

    # 4. Stack, move back to cpu and return everything
    predictions = torch.stack(predictions).cpu()
    confidence_levels = torch.stack(confidence_levels).cpu()
    true_sentiments = torch.stack(true_sentiments).cpu()
    
    return reviews, predictions, confidence_levels, true_sentiments

This is similar to the evaluation function, except ...

In [55]:
test_reviews, test_predictions, test_confidence_lvls, test_sentiments = get_predictions_from_dl(classifier, test_dl)

100%|██████████| 698/698 [02:19<00:00,  4.99it/s]


Let's have a look at the classification report

In [58]:
classification_report(test_sentiments, test_predictions)

'              precision    recall  f1-score   support\n\n           0       0.79      0.78      0.78      3706\n           1       0.70      0.72      0.71      3785\n           2       0.89      0.88      0.88      3666\n\n    accuracy                           0.79     11157\n   macro avg       0.79      0.79      0.79     11157\nweighted avg       0.79      0.79      0.79     11157\n'

In [60]:
idx = 57

review = test_reviews[idx]
confidence_lvls = test_confidence_lvls[idx]

print(review)
conf_lvls = {number2sentiment[idx] : confidence_lvls[idx] for idx in range(3)}
conf_lvls

The price is good, size is perfect...but my dog doesn't like them at all. Would not buy again sorry. Not the products fault


{'negative': tensor(0.7084),
 'neutral': tensor(0.2869),
 'positive': tensor(0.0047)}

### Predicting on Raw Text

In [94]:
Text = r"""Very easy game. Explained in a minute. Anyone can play it. Very fast laps. The limit could be another two or three centimeters shorter,
then it will “click” faster! I did it this way. It can be packed up to a small size and is almost indestructible.
But it can easily lead to frustration if you play with too much ambition. You also have to be able to lose!"""

In [95]:
score = 4

In [96]:
encoding = tokenizer.encode_plus(Text, add_special_tokens=True, max_length=512,
                                 padding='max_length', truncation=True,
                                 return_token_type_ids=False, return_tensors='pt')

In [97]:
input_ids = encoding['input_ids'].to(device)
attention_mask = encoding['attention_mask'].to(device)

pooled = classifier(input_ids, attention_mask)

confidence = F.softmax(pooled, dim=1).tolist()[0]
confidence = {number2sentiment[idx] : confidence[idx] for idx in range(3)}

val, prediction = torch.max(pooled, dim=1)
prediction = number2sentiment[prediction.item()]

print(confidence)
prediction

{'negative': 0.001904321019537747, 'neutral': 0.023873839527368546, 'positive': 0.97422194480896}


'positive'

## Summary

## References

- [Huggingface Transformers](https://huggingface.co/transformers/)
- 